In [1]:
import os, tarfile
import re, glob
import pandas as pd
import numpy as np
import zipfile, gzip
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

In [2]:
os.chdir("/data/projects/DNABERT_data/Core_promoters/Core_promoter_regions")

In [3]:
file_extension = '.csv'
all_filenames = [i for i in glob.glob(f"*{file_extension}")]

In [4]:
all_filenames

['chr11_core_promoter_sequence.csv',
 'chr4_core_promoter_sequence.csv',
 'chr20_core_promoter_sequence.csv',
 'chr15_core_promoter_sequence.csv',
 'chr7_core_promoter_sequence.csv',
 'chr12_core_promoter_sequence.csv',
 'chr3_core_promoter_sequence.csv',
 'chr10_core_promoter_sequence.csv',
 'chrX_core_promoter_sequence.csv',
 'chr21_core_promoter_sequence.csv',
 'chr14_core_promoter_sequence.csv',
 'chr16_core_promoter_sequence.csv',
 'chr1_core_promoter_sequence.csv',
 'chr6_core_promoter_sequence.csv',
 'chr17_core_promoter_sequence.csv',
 'chr8_core_promoter_sequence.csv',
 'chr19_core_promoter_sequence.csv',
 'chr5_core_promoter_sequence.csv',
 'chr18_core_promoter_sequence.csv',
 'chr2_core_promoter_sequence.csv',
 'chr9_core_promoter_sequence.csv',
 'chr22_core_promoter_sequence.csv',
 'chrY_core_promoter_sequence.csv',
 'chr13_core_promoter_sequence.csv']

In [5]:
combined_csv_data = pd.concat([pd.read_csv(f) for f in all_filenames])
combined_csv_data.drop_duplicates().reset_index(drop=True)

,gene_id,transcript_id,strand,exon_id,transcript_type,TSS,sequence
0,ENSG00000173153.17,ENST00000000442.11,+,ENSE00001884684.2,protein_coding,64305524,GCTTTGCATGCGCACGGCCGGCCCCACCCCCGCTGTCAGCTGGAGG...
1,ENSG00000048028.11,ENST00000003302.8,-,ENSE00003474148.1,protein_coding,113875570,GGACGGAGCGGGGTGGAGGGCGGGGGCGGGGCCTAGTCCTGAGAGG...
2,ENSG00000005801.18,ENST00000005082.13,-,ENSE00002185298.1,protein_coding,3379116,CTCCACTTCCTGTCCCCGCGGCTCTGTGTCCCCTGCTAGCCGTAGG...
3,ENSG00000006611.17,ENST00000005226.12,-,ENSE00002182900.2,protein_coding,17544416,TGGCCCCGCCCCTCCGCGCTCAGGCCCCGCCCCCAGCTCCGAGGGC...
4,ENSG00000006118.15,ENST00000005286.8,+,ENSE00002270579.1,protein_coding,60924481,TCCCTCCCACCCCACTGCTCCCGCTCCATTGTCTGGGAATTgcagc...
...,...,...,...,...,...,...,...
236970,ENSG00000005810.19,ENST00000684354.1,-,ENSE00003848740.1,protein_coding,77327094,GTGCGAGGAGTGGCGCATGCTCTGTGAGGCCGGCAGCTTCCCATTG...
236971,ENSG00000151835.17,ENST00000684385.1,-,ENSE00003918114.1,protein_coding,23433752,ATGACAGTCCTGTCTCTTTAAGAAGAAAAAAAAAGGCTGCTGGGTA...
236972,ENSG00000132932.19,ENST00000684424.1,+,ENSE00003917000.1,protein_coding,25468838,cgcggcacaggcggcggcgTCTCCAGGGGGAGCCAAGGTACGTAGG...
236973,ENSG00000151835.17,ENST00000684497.1,-,ENSE00003918114.1,protein_coding,23433752,ATGACAGTCCTGTCTCTTTAAGAAGAAAAAAAAAGGCTGCTGGGTA...


In [6]:
df_pos=pd.DataFrame()
df_pos['Sequence'] = combined_csv_data['sequence'].str.upper()

In [7]:
df_pos

,Sequence
0,GCTTTGCATGCGCACGGCCGGCCCCACCCCCGCTGTCAGCTGGAGG...
1,GGACGGAGCGGGGTGGAGGGCGGGGGCGGGGCCTAGTCCTGAGAGG...
2,CTCCACTTCCTGTCCCCGCGGCTCTGTGTCCCCTGCTAGCCGTAGG...
3,TGGCCCCGCCCCTCCGCGCTCAGGCCCCGCCCCCAGCTCCGAGGGC...
4,TCCCTCCCACCCCACTGCTCCCGCTCCATTGTCTGGGAATTGCAGC...
...,...
4320,GTGCGAGGAGTGGCGCATGCTCTGTGAGGCCGGCAGCTTCCCATTG...
4321,ATGACAGTCCTGTCTCTTTAAGAAGAAAAAAAAAGGCTGCTGGGTA...
4322,CGCGGCACAGGCGGCGGCGTCTCCAGGGGGAGCCAAGGTACGTAGG...
4323,ATGACAGTCCTGTCTCTTTAAGAAGAAAAAAAAAGGCTGCTGGGTA...


In [8]:
# TATAWAWR (W = A or T, R = A or G)



# TATAAAA
# TATAAAT
# TATATAA
# TATATAT


substrings = ['TATAAAA', 'TATAAAT', 'TATATAA', 'TATAAAA']
rev_com = ['TTTTATA', 'ATTTATA', 'TTATATA', 'TTTTATA']
#substrings =['TATA']

In [9]:
df_TATA = df_pos[df_pos['Sequence'].apply(lambda x: any(substring in x for substring in substrings))]

In [10]:
df_TATA

,Sequence
5,ACCTATAAAATCCAGGTAAATATCTGATACTGGCACACAGGTTGGA...
44,ATGATATAAAACAGGTCAGAACCCTCCTGCCTGTCTGCTCAGTTCA...
65,GATATAAAAGTGCTCCAACAAACCCGGAGCTGGCGTGCCGACCGCG...
86,TTTGTATAAAAGGTGAACTGAGATTTCATTCAGTCTACAGCTCTTG...
103,GCTTTTCTTCCCTCCCAAGAGACCAGCAAGGCTCACTATAAATAGC...
...,...
4002,CCATCTATAAATAGCTGGTCATCAGGGTTGATGAGAGGCTCGGAGG...
4090,CTTATAAAAGGGACCCCAGGGAGGTAGCCCCTTCTACCATGTGTGG...
4131,TTATAAAAGGGACCCCAGGGAGGTAGCCCCTTCTACCATGTGTGGA...
4139,TTTGCTAGCATATAAATCTCTCGGAGGCAGAAAACATGTCTGAAAT...


In [19]:
df_non_TATA = df_pos[~df_pos['Sequence'].apply(lambda x: any(substring in x for substring in substrings))]

In [20]:
df_non_TATA

,Sequence
0,GCTTTGCATGCGCACGGCCGGCCCCACCCCCGCTGTCAGCTGGAGG...
1,GGACGGAGCGGGGTGGAGGGCGGGGGCGGGGCCTAGTCCTGAGAGG...
2,CTCCACTTCCTGTCCCCGCGGCTCTGTGTCCCCTGCTAGCCGTAGG...
3,TGGCCCCGCCCCTCCGCGCTCAGGCCCCGCCCCCAGCTCCGAGGGC...
4,TCCCTCCCACCCCACTGCTCCCGCTCCATTGTCTGGGAATTGCAGC...
...,...
4319,ATGACAGTCCTGTCTCTTTAAGAAGAAAAAAAAAGGCTGCTGGGTA...
4320,GTGCGAGGAGTGGCGCATGCTCTGTGAGGCCGGCAGCTTCCCATTG...
4321,ATGACAGTCCTGTCTCTTTAAGAAGAAAAAAAAAGGCTGCTGGGTA...
4322,CGCGGCACAGGCGGCGGCGTCTCCAGGGGGAGCCAAGGTACGTAGG...


In [7]:
df_pos=pd.DataFrame()
df_pos['Sequence'] = combined_csv_data['sequence'].str.upper().apply(seq2kmer, args=(6,))

In [8]:
df_pos

,Sequence
0,GCTTTG CTTTGC TTTGCA TTGCAT TGCATG GCATGC CATG...
1,GGACGG GACGGA ACGGAG CGGAGC GGAGCG GAGCGG AGCG...
2,CTCCAC TCCACT CCACTT CACTTC ACTTCC CTTCCT TTCC...
3,TGGCCC GGCCCC GCCCCG CCCCGC CCCGCC CCGCCC CGCC...
4,TCCCTC CCCTCC CCTCCC CTCCCA TCCCAC CCCACC CCAC...
...,...
4320,GTGCGA TGCGAG GCGAGG CGAGGA GAGGAG AGGAGT GGAG...
4321,ATGACA TGACAG GACAGT ACAGTC CAGTCC AGTCCT GTCC...
4322,CGCGGC GCGGCA CGGCAC GGCACA GCACAG CACAGG ACAG...
4323,ATGACA TGACAG GACAGT ACAGTC CAGTCC AGTCCT GTCC...


In [9]:
df_pos['Label']=1

In [10]:
df_pos = df_pos.reset_index(drop=True)
df_pos

,Sequence,Label
0,GCTTTG CTTTGC TTTGCA TTGCAT TGCATG GCATGC CATG...,1
1,GGACGG GACGGA ACGGAG CGGAGC GGAGCG GAGCGG AGCG...,1
2,CTCCAC TCCACT CCACTT CACTTC ACTTCC CTTCCT TTCC...,1
3,TGGCCC GGCCCC GCCCCG CCCCGC CCCGCC CCGCCC CGCC...,1
4,TCCCTC CCCTCC CCTCCC CTCCCA TCCCAC CCCACC CCAC...,1
...,...,...
236970,GTGCGA TGCGAG GCGAGG CGAGGA GAGGAG AGGAGT GGAG...,1
236971,ATGACA TGACAG GACAGT ACAGTC CAGTCC AGTCCT GTCC...,1
236972,CGCGGC GCGGCA CGGCAC GGCACA GCACAG CACAGG ACAG...,1
236973,ATGACA TGACAG GACAGT ACAGTC CAGTCC AGTCCT GTCC...,1


In [12]:
df_pos.to_csv("/data/projects/DNABERT_data/Core_promoters/Core_promoter_regions/positive_set.tsv", sep="\t", index=False)